In [6]:
# initializing

import pandas as pd
import pandas_gbq as pgbq
from multiprocessing import Pool
from scipy import stats
import os
import time

directory = '/Users/joshharris/community_health/sql/community_facts/'
google_project_id = 'solid-ridge-104914'
bq_dataset = 'community_networks'

In [23]:
def sql_module_worker(worker_args):
    directory = worker_args[0]
    bq_dataset = worker_args[1]
    sql_modules = worker_args[2]
    i = worker_args[3]
    sql_module_type = os.path.basename(directory)
    sql_module = sql_modules[i]
    print('Starting module ',i,': ',sql_module, sep='')
    try:
        result = pgbq.read_gbq('select * from '+bq_dataset+'.'+sql_module_type, google_project_id, dialect='standard', verbose=False)
    except:
        print(sql_module_type+' table does not exist, generating now.')
        with open(directory+sql_module) as query_file:
            query = query_file.read()
        result = pgbq.read_gbq(query, google_project_id, dialect='standard', verbose=False)
        print('Saving '+sql_module_type+'_'+sql_module+' table')
        result.to_gbq(result, bq_dataset+'.'+sql_module_type+'_'+sql_module, google_project_id, if_exists='replace', verbose=False)
    return (result) 
        

In [24]:
#Run network sql modules and join with community facts

def build_stats_table(directory, bq_dataset):
    
    sql_modules = list(map(lambda x: os.path.splitext(x)[0], os.listdir(directory)))
    worker_args = [directory, bq_dataset, sql_modules, range(len(sql_modules))]
    if __name__ == '__main__':
        with Pool(processes=len(sql_modules)) as pool:
            result, sql_module_type = pool.map(sql_module_worker, (worker_args,))
            pool.close()
            pool.join()

    print('Joining results')
    stats_table = result[0].set_index('cid')
    for i in range(len(result)-1):
        stats_table = stats_table.join(result[i+1].set_index('cid'))
    stats_table.to_gbq(stats_table, bq_dataset+'.'+sql_module_type, google_project_id, if_exists='replace', verbose=False)
    print('\n\nDone\n\n', sep='')

In [25]:
build_stats_table(community_facts_directory, bq_dataset)

TypeError: list indices must be integers or slices, not range